In [1]:
import pandas as pd
import numpy as np
table1 = pd.read_csv('./table1.csv')
table2 = pd.read_csv('./table2.csv')

In [29]:
total_region = pd.concat([table1, table2], axis=0)
total_MHC_region = total_region[total_region['MHC region?']=='Yes'].reset_index(drop=False)
total_MHC_region['-log10(p)_pvalues'] = 0
total_MHC_region['Dprime_pvalues'] = 0
total_MHC_region['R2_pvalues'] = 0
from scipy import stats
for race in total_MHC_region['Race'].unique():
    pop_MHC_df = total_MHC_region[(total_MHC_region['Race']==race)]
    pop_ld = pd.read_csv(f'./LD_results/MHC_{race}.tsv', sep='\t').drop_duplicates()
    pop_ld = pop_ld[pop_ld['RS_Number'] != 'RS_Number']
    pop_ld['pos'] = pop_ld['Coord'].str.split(':').str[1].astype(int)
    pop_ld['P-value'] = pop_ld['P-value'].astype(float)
    pop_ld['-log10(p)'] = -np.log(pop_ld['P-value'])
    pop_ld['Dprime'] = pop_ld['Dprime'].astype(float)
    pop_ld['R2'] = pop_ld['R2'].astype(float)
    pop_ld['region'] = 'unselected'
    pop_ld['Race'] = race
    for region in pop_MHC_df[['Start', 'End']].to_numpy():
        start, end = region
        region_id = '_'.join([str(start), str(end)])
        pop_ld.loc[(pop_ld['pos'] >= start) & (pop_ld['pos'] <= end), 'region'] = region_id
    neg_logps_pvalues = []
    dprimes_pvalues = []
    r2s_pvalues = []   
    for region in pop_MHC_df[['Start', 'End']].to_numpy():
        start, end = region
        region_id = '_'.join([str(start), str(end)])
        # compare the distribution of p-values in the region and the unselected region, to see if the region is enriched for high p-values, with Mann-Whitney U test
        region_df = pop_ld[pop_ld['region'] == region_id]
        unselected_df = pop_ld[pop_ld['region'] == 'unselected']
        pvalue_nlogp = stats.mannwhitneyu(region_df['-log10(p)'], unselected_df['-log10(p)'], alternative='greater')[1]
        pvalue_dprime = stats.mannwhitneyu(region_df['Dprime'], unselected_df['Dprime'], alternative='greater')[1]
        pvalue_r2 = stats.mannwhitneyu(region_df['R2'], unselected_df['R2'], alternative='greater')[1]
        # if pvalue_dprime < 0.05:
        #     print(region_df[['-log10(p)', 'Dprime', 'R2']].mean())
        #     print(unselected_df[['-log10(p)', 'Dprime', 'R2']].mean())
        neg_logps_pvalues.append(pvalue_nlogp)
        dprimes_pvalues.append(pvalue_dprime)
        r2s_pvalues.append(pvalue_r2)
    total_MHC_region.loc[pop_MHC_df.index ,'-log10(p)_pvalues'] = neg_logps_pvalues
    total_MHC_region.loc[pop_MHC_df.index ,'Dprime_pvalues'] = dprimes_pvalues
    total_MHC_region.loc[pop_MHC_df.index ,'R2_pvalues'] = r2s_pvalues
from statsmodels.stats.multitest import multipletests
total_MHC_region['-log10(p)_qvalues'] = multipletests(total_MHC_region['-log10(p)_pvalues'], method='fdr_bh')[1]
total_MHC_region['Dprime_qvalues'] = multipletests(total_MHC_region['Dprime_pvalues'], method='fdr_bh')[1]
total_MHC_region['R2_qvalues'] = multipletests(total_MHC_region['R2_pvalues'], method='fdr_bh')[1]
filtered_MHC_region = total_MHC_region[~((total_MHC_region['-log10(p)_qvalues'] < 0.05) | (total_MHC_region['Dprime_qvalues'] < 0.05) | (total_MHC_region['R2_qvalues'] < 0.05))]
print(f'Removed {total_MHC_region.shape[0] - filtered_MHC_region.shape[0]} regions')


Removed 15 regions


In [24]:
removed_regions = total_MHC_region[((total_MHC_region['-log10(p)_qvalues'] < 0.05) | (total_MHC_region['Dprime_qvalues'] < 0.05) | (total_MHC_region['R2_qvalues'] < 0.05))]
removed_regions
                                    

,index,Race,Chromosome,Start,End,Length,MHC region?,Average of Pd,Standard deviation of Pd,p-value,Corrected p-value (FDR/BH method),Average of Pa,Standard deviation of Pa,Corrected p-value (FDR/BH method),-log10(p)_pvalues,Dprime_pvalues,R2_pvalues,-log10(p)_qvalues,Dprime_qvalues,R2_qvalues
3,3,All,6,29720403,29913914,193511,Yes,0.133014,0.155387,6.200000e-14,2.520000e-13,NaN,NaN,NaN,0.917632,1.985072e-03,1.253163e-02,0.999819,0.016079,5.970955e-02
6,6,ASW,6,29720403,30120966,400563,Yes,0.177049,0.228807,5.000000e-15,2.900000e-14,NaN,NaN,NaN,0.829676,2.328964e-01,2.497898e-04,0.999819,0.992874,1.686081e-03
12,12,CDX,6,30994370,31528792,534422,Yes,0.144770,0.208126,6.810000e-12,1.730000e-11,NaN,NaN,NaN,0.983620,1.237236e-03,3.163217e-03,0.999819,0.012527,1.970928e-02
33,33,GIH,6,29720403,29913914,193511,Yes,0.112729,0.145921,1.450000e-06,2.180000e-06,NaN,NaN,NaN,0.915226,4.207263e-04,4.242333e-05,0.999819,0.004868,3.123900e-04
36,36,GWD,6,29720403,30120966,400563,Yes,0.192514,0.246566,0.000000e+00,0.000000e+00,NaN,NaN,NaN,0.736447,1.281190e-04,2.766430e-05,0.999819,0.001730,2.240808e-04
42,42,ITU,6,29720403,30011739,291336,Yes,0.117442,0.153762,1.170000e-07,1.900000e-07,NaN,NaN,NaN,0.453397,2.006886e-07,1.508952e-07,0.999819,0.000015,3.435143e-06
45,45,JPT,6,29720403,29913914,193511,Yes,0.155029,0.201899,1.560000e-13,5.630000e-13,NaN,NaN,NaN,0.771813,1.135982e-04,2.145174e-07,0.999819,0.001730,3.445879e-06
46,46,JPT,6,29939668,30120966,181298,Yes,0.145316,0.198916,9.710000e-12,2.320000e-11,NaN,NaN,NaN,0.479927,1.696431e-01,1.364846e-09,0.999819,0.775857,1.105525e-07
47,47,JPT,6,30994370,31528792,534422,Yes,0.121645,0.169902,5.810000e-08,9.840000e-08,NaN,NaN,NaN,0.987132,8.237505e-01,1.362028e-05,0.999819,1.000000,1.225825e-04
49,49,KHV,6,29720403,30120966,400563,Yes,0.161858,0.214232,9.000000e-15,4.580000e-14,NaN,NaN,NaN,0.910059,5.537145e-07,3.376133e-07,0.999819,0.000015,3.906668e-06


In [21]:
import os
non_MHC_region = total_region[total_region['MHC region?']!='Yes'].reset_index(drop=False)
non_MHC_region
background_region = []
for region in non_MHC_region[['Race', 'Chromosome', 'Start', 'End']].to_numpy():
    region_index = '_'.join([str(x) for x in region]    )
    region_mid = int((int(region[2]) + int(region[3])) / 2)
    if region_mid-1500000 < 0:
        background_region.append([region[0], region[1], 0, 3000000, region_index])
    else:
        background_region.append([region[0], region[1], region_mid-1500000, region_mid+1500000-1, region_index])
background_region = pd.DataFrame(background_region, columns=['Race', 'Chromosome', 'Start', 'End', 'ref_for'])
background_region['is_background'] = 'ref'
background_region['filename'] = background_region['Race'].astype(str) + '_' + background_region['Chromosome'].astype(str) + '_' + background_region['Start'].astype(str) + '_' + background_region['End'].astype(str)+ '_' + background_region['is_background'].astype(str)+ ".tsv"
import os
neg_logps_pvalues = []
dprimes_pvalues = []
r2s_pvalues = []
for region in background_region.to_numpy():
    race, chrom, start, end, ref_for, _, fname = region
    pvalue_nlogp = 1
    pvalue_dprime = 1
    pvalue_r2 = 1
    if os.path.exists(f'./LD_results/LD_analysis_results/{fname}'):
        pop_ld = pd.read_csv(f'./LD_results/LD_analysis_results/{fname}', sep='\t')
        pop_ld['pos'] = pop_ld['Coord'].str.split(':').str[1].astype(int)
        pop_ld['chr'] = pop_ld['Coord'].str.split(':').str[0]
        pop_ld['pos'] = pop_ld['Coord'].str.split(':').str[1].astype(int)
        pop_ld['P-value'] = pop_ld['P-value'].astype(float)
        pop_ld['-log10(p)'] = -np.log(pop_ld['P-value'])
        pop_ld['Dprime'] = pop_ld['Dprime'].astype(float)
        pop_ld['R2'] = pop_ld['R2'].astype(float)
        actual_start, actual_end = [int(x) for x in ref_for.split('_')[2:4]]
        unselected_df = pop_ld[(pop_ld['pos'] < actual_start) | (pop_ld['pos'] > actual_end)]
        region_df = pop_ld[(pop_ld['pos'] >= actual_start) & (pop_ld['pos'] <= actual_end)]
        if len(region_df) != 0:
            pvalue_nlogp = stats.mannwhitneyu(region_df['-log10(p)'], unselected_df['-log10(p)'], alternative='greater')[1]
            pvalue_dprime = stats.mannwhitneyu(region_df['Dprime'], unselected_df['Dprime'], alternative='greater')[1]
            pvalue_r2 = stats.mannwhitneyu(region_df['R2'], unselected_df['R2'], alternative='greater')[1]
    neg_logps_pvalues.append(pvalue_nlogp)
    dprimes_pvalues.append(pvalue_dprime)
    r2s_pvalues.append(pvalue_r2)
non_MHC_region['-log10(p)_pvalues'] = neg_logps_pvalues
non_MHC_region['Dprime_pvalues'] = dprimes_pvalues
non_MHC_region['R2_pvalues'] = r2s_pvalues
neg_logps_qvalues = multipletests(non_MHC_region['-log10(p)_pvalues'], method='fdr_bh')[1]
dprimes_qvalues = multipletests(non_MHC_region['Dprime_pvalues'], method='fdr_bh')[1]
r2s_qvalues = multipletests(non_MHC_region['R2_pvalues'], method='fdr_bh')[1]
non_MHC_region['-log10(p)_qvalues'] = neg_logps_qvalues
non_MHC_region['Dprime_qvalues'] = dprimes_qvalues
non_MHC_region['R2_qvalues'] = r2s_qvalues
filtered_non_MHC_region = non_MHC_region[~((non_MHC_region['-log10(p)_qvalues'] < 0.05) | (non_MHC_region['Dprime_qvalues'] < 0.05) | (non_MHC_region['R2_qvalues'] < 0.05))]
print(f'Removed {non_MHC_region.shape[0] - filtered_non_MHC_region.shape[0]} regions')



Removed 43 regions
